In [ ]:
!pip install --quiet flask moviepy whisper-timestamped pydub srt ffmpeg-python numpy==1.26.4
!pip install --quiet pyngrok

!apt-get update -qq
!apt-get install -qq -y ffmpeg

print("All dependencies installed successfully.")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 15.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.7/801.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 1. Uninstall existing numpy and numba to start clean
!pip uninstall -y numpy numba

# 2. Install a compatible version of numpy first
!pip install numpy==1.26.4

# 3. Now install all other libraries. pip will find a compatible numba version
!pip install moviepy whisper-timestamped pydub srt ffmpeg-python flask pyngrok

# 4. Ensure FFmpeg is available
!apt-get update -qq
!apt-get install -qq -y ffmpeg

print("All dependencies installed successfully.")

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: numba 0.60.0
Uninstalling numba-0.60.0:
  Successfully uninstalled numba-0.60.0
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai-whisper 20250625 requires numba, which is not installed.
stumpy 1.13.0 requires numba>=0.57.1, which is not installed.
cuml-cu12 25.6.0 requires numba<0.62.0a0,>=0.59.1, which is not installed.
umap-learn 0.5.9.post2 requires numba>=0.51.2, which is not installed.
dask-cuda 25.6.0 requires numba<0.62.0a0,>=0.59.1, which is not installed.
shap 0.48.0 requires numba>=0.54, which is not installed.
cud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
All dependencies installed successfully.


In [ ]:
import os
import uuid
import threading
import time
from flask import Flask, jsonify, send_from_directory, request, render_template_string
from werkzeug.utils import secure_filename
import moviepy.editor as mp
import whisper_timestamped as whisper
from pydub import AudioSegment
from google.colab import drive

# --- Configuration ---
UPLOAD_FOLDER = '/content/uploads'
OUTPUT_FOLDER = '/content/outputs'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['OUTPUT_FOLDER'] = OUTPUT_FOLDER

# --- Processing Functions ---
def get_diarization(audio_file_path):
    print("Starting audio diarization...")
    audio = whisper.load_audio(audio_file_path)
    model = whisper.load_model("tiny", device="cpu")
    result = whisper.transcribe(model, audio, language="en", diarization=True)

    segments = []
    for segment in result["segments"]:
        segments.append({
            "speaker": segment.get("speaker", "UNKNOWN"),
            "start": segment["start"],
            "end": segment["end"]
        })
    print("Diarization complete.")
    return segments

def create_edited_video(video1_path, video2_path, diarization_output, output_path, job_id, srt_path):
    try:
        print(f"Starting video editing for job {job_id}...")
        video1_clip = mp.VideoFileClip(video1_path)
        video2_clip = mp.VideoFileClip(video2_path)

        speakers = sorted(list(set(s['speaker'] for s in diarization_output)))
        speaker_map = {}
        if len(speakers) >= 2:
            speaker_map[speakers[0]] = video1_clip
            speaker_map[speakers[1]] = video2_clip
        else:
            print("Warning: Could not identify two speakers. Using a single video source.")
            speaker_map = {s['speaker']: video1_clip for s in diarization_output}

        print(f"Speaker map: {speaker_map}")

        final_clips = []
        last_end_time = 0

        for segment in diarization_output:
            start = segment['start']
            end = segment['end']
            speaker = segment['speaker']

            if start > last_end_time + 0.1 and len(speakers) >= 2:
                final_clips.append(video2_clip.subclip(last_end_time, start))

            if speaker in speaker_map:
                clip = speaker_map[speaker].subclip(start, end)
                final_clips.append(clip)

            last_end_time = end

        if not final_clips:
            print("No speaker segments found. Creating a blank video.")
            total_duration = video1_clip.duration
            final_video = video1_clip.subclip(0, total_duration)
        else:
            final_video = mp.concatenate_videoclips(final_clips, method="compose")

        final_video.write_videofile(output_path, codec="libx264", audio_codec="aac")

        video1_clip.close()
        video2_clip.close()
        print(f"Final video saved to {output_path}")
    except Exception as e:
        print(f"An error occurred during video editing for job {job_id}: {e}")

# --- Flask Routes ---
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Podcast Video Editor</title>
    <style>
        body { font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif; line-height: 1.6; background-color: #f0f2f5; color: #333; margin: 0; padding: 2em; }
        .container { max-width: 800px; margin: auto; background-color: #fff; padding: 2em; border-radius: 8px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); }
        h1 { color: #222; border-bottom: 2px solid #3498db; padding-bottom: 0.5em; }
        button { background-color: #3498db; color: #fff; border: none; padding: 0.75em 1.5em; font-size: 1em; border-radius: 4px; cursor: pointer; transition: background-color 0.3s ease; }
        button:hover { background-color: #2980b9; }
        #status { margin-top: 2em; padding: 1.5em; border: 1px solid #ddd; background-color: #f9f9f9; border-radius: 4px; }
        #status p { margin: 0.5em 0; }
        #status a { color: #3498db; text-decoration: none; font-weight: bold; }
        .hidden { display: none; }
    </style>
</head>
<body>
    <div class="container">
        <h1>🎙️ Podcast Video Editor</h1>
        <p>Upload your video and audio files below to generate an edited video.</p>
        <form id="upload-form" enctype="multipart/form-data">
            <div><label>Podcast Audio (.mp3): <input type="file" name="audio" required></label></div>
            <div><label>Video 1 (.mp4): <input type="file" name="video1" required></label></div>
            <div><label>Video 2 (.mp4): <input type="file" name="video2" required></label></div>
            <div><label>Subtitle File (.srt): <input type="file" name="srt" required></label></div>
            <button type="submit">Start Processing</button>
        </form>
        <div id="status"></div>
    </div>
    <script>
        document.getElementById('upload-form').addEventListener('submit', function(e) {
            e.preventDefault();
            var form = e.target;
            var formData = new FormData(form);
            var statusDiv = document.getElementById('status');
            statusDiv.innerHTML = '<h2>Processing Status</h2><p>Uploading files...</p>';

            fetch('/upload', { method: 'POST', body: formData })
            .then(response => response.json())
            .then(data => {
                if (data.job_id) {
                    statusDiv.innerHTML += '<p>Processing started. Job ID: ' + data.job_id + '</p>';
                    checkStatus(data.job_id, statusDiv);
                } else {
                    statusDiv.innerHTML = '<p style="color:red;">Error: ' + data.error + '</p>';
                }
            })
            .catch(error => {
                statusDiv.innerHTML = '<p style="color:red;">An error occurred. Please try again.</p>';
                console.error('Error:', error);
            });
        });

        function checkStatus(jobId, statusDiv) {
            var statusInterval = setInterval(function() {
                fetch('/status/' + jobId)
                .then(response => response.json())
                .then(data => {
                    if (data.status === 'completed') {
                        clearInterval(statusInterval);
                        statusDiv.innerHTML += '<p>Processing complete! <a href="' + data.video_url + '">Download Final Video</a></p>';
                    } else {
                        statusDiv.innerHTML = '<h2>Processing Status</h2><p>Processing...</p>';
                    }
                })
                .catch(error => {
                    clearInterval(statusInterval);
                    statusDiv.innerHTML = '<p style="color:red;">Error checking status.</p>';
                });
            }, 5000);
        }
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/upload', methods=['POST'])
def upload_files():
    if not all(k in request.files for k in ['audio', 'video1', 'video2', 'srt']):
        return jsonify({'error': 'Missing one or more required files.'}), 400

    job_id = str(uuid.uuid4())
    job_dir = os.path.join(app.config['UPLOAD_FOLDER'], job_id)
    os.makedirs(job_dir, exist_ok=True)

    file_paths = {}
    for key, file in request.files.items():
        filename = secure_filename(file.filename)
        path = os.path.join(job_dir, filename)
        file.save(path)
        file_paths[key] = path

    thread = threading.Thread(target=process_job, args=(job_id, file_paths))
    thread.start()

    return jsonify({'job_id': job_id, 'message': 'Processing started.'}), 202

def process_job(job_id, file_paths):
    try:
        diarization_output = get_diarization(file_paths['audio'])
        output_video_path = os.path.join(app.config['OUTPUT_FOLDER'], f'podcast_edited_{job_id}.mp4')
        create_edited_video(file_paths['video1'], file_paths['video2'], diarization_output, output_video_path, job_id, file_paths['srt'])
    except Exception as e:
        print(f"Error processing job {job_id}: {e}")

@app.route('/status/<job_id>', methods=['GET'])
def get_status(job_id):
    output_video_path = os.path.join(app.config['OUTPUT_FOLDER'], f'podcast_edited_{job_id}.mp4')
    if os.path.exists(output_video_path):
        return jsonify({'status': 'completed', 'video_url': f'/download/{job_id}'}), 200
    return jsonify({'status': 'processing'}), 200

@app.route('/download/<job_id>')
def download_file(job_id):
    output_video_path = os.path.join(app.config['OUTPUT_FOLDER'], f'podcast_edited_{job_id}.mp4')
    if not os.path.exists(output_video_path):
        return jsonify({'error': 'File not found.'}), 404

    return send_from_directory(app.config['OUTPUT_FOLDER'], f'podcast_edited_{job_id}.mp4', as_attachment=True)

# Start Flask app in a separate thread
def run_flask_app():
    app.run(host='0.0.0.0', port=5000)

threading.Thread(target=run_flask_app).start()

print("Flask app is starting...")

 * Serving Flask app '__main__'
 * Debug mode: off
Flask app is starting...


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [ ]:
from pyngrok import ngrok

# --- Add your ngrok auth token here ---
ngrok.set_auth_token("30x94ESQqCXrkl21X2ZlDun6PIH_2762NeGMLiKkJJ47vDVnu")

# Connect to the Flask app on port 5000
public_url = ngrok.connect(addr=5000, proto="http")

print("Public URL:", public_url)
print("Click the URL above to access your Flask application.")

Public URL: NgrokTunnel: "https://095c06bfbc6a.ngrok-free.app" -> "http://localhost:5000"
Click the URL above to access your Flask application.
